In [ ]:
import requests
import json
import pymysql
import time
import traceback
def getdata():#从腾讯接口爬取数据，并存到字典及列表中
    url = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3947.100 Safari/537.36"
    }
    r1 = requests.get(url,headers = header)
    res1 = json.loads(r1.text)
    data_all = json.loads(res1["data"])
    
    url2 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
    r2 = requests.get(url2,headers = header)
    res2 = json.loads(r2.text)
    odata_all = json.loads(res2["data"])
    
    country_history = {}#历史数据，全国
    for i in odata_all["chinaDayList"]:
        ds ="2020."+ i["date"]
        tup =time.strptime(ds,"%Y.%m.%d")
        ds =time.strftime("%Y-%m-%d",tup)
        confirm =i["confirm"]
        heal=i["heal"]
        dead=i["dead"]
        country_history[ds] ={"confirm":confirm, "heal":heal, "dead":dead}
    for i in odata_all["chinaDayAddList"]:
        ds ="2020."+ i["date"]
        tup =time.strptime(ds,"%Y.%m.%d")
        ds =time.strftime("%Y-%m-%d",tup)
        confirm =i["confirm"]
        country_history[ds].update({"confirm_add":confirm})
        
    details =[]#各省数据
    update_time=data_all["lastUpdateTime"]
    data_province=data_all["areaTree"][0]["children"]
    for pro_infos in data_province:
        province=pro_infos["name"]
        confirm=pro_infos["total"]["confirm"]
        confirm_add=pro_infos["today"]["confirm"]
        heal=pro_infos["total"]["heal"]
        dead=pro_infos["total"]["dead"]
        details.append([update_time,province,confirm,confirm_add,heal,dead])
    
    country_now =[]#全国如今数据
    update_time =data_all["lastUpdateTime"]
    nowConfirm =data_all["chinaTotal"]["nowConfirm"]
    suspect =data_all["chinaTotal"]["suspect"]
    nowSevere =data_all["chinaTotal"]["nowSevere"]
    confirm =data_all["chinaTotal"]["confirm"]
    heal =data_all["chinaTotal"]["heal"]
    dead =data_all["chinaTotal"]["dead"]    
    nowConfirm_add =data_all["chinaAdd"]["nowConfirm"]
    suspect_add =data_all["chinaAdd"]["suspect"]
    nowSevere_add =data_all["chinaAdd"]["nowSevere"]
    confirm_add =data_all["chinaAdd"]["confirm"]
    heal_add =data_all["chinaAdd"]["heal"]
    dead_add =data_all["chinaAdd"]["dead"]
    country_now.append([update_time,nowConfirm,suspect,nowSevere,confirm,heal,dead,nowConfirm_add,suspect_add,nowSevere_add,confirm_add,heal_add,dead_add])
    
    province_history = []#历史数据，各省
    ds=time.strftime("%Y-%m-%d")
    data_province=data_all["areaTree"][0]["children"]
    for pro_infos in data_province:
        province=pro_infos["name"]
        confirm=pro_infos["total"]["confirm"]
        confirm_add=pro_infos["today"]["confirm"]
        heal=pro_infos["total"]["heal"]
        dead=pro_infos["total"]["dead"]
        province_history.append([ds,province,confirm,confirm_add,heal,dead])
    
    return country_history,details,country_now,province_history